1. Спарсите спам из ваших почтовых ящиков в текстовые файлы
2. Выберите наиболее часто встречающие слова в спаме. Длина слова - не менее 5 букв. Из полученного списка удалите прилагательные. Выберите из оставшихся 5-7 слов - они будут индикаторами.
3. Напишите классификатор спама на основе выбранных слов-индикаторов. Используйте наивный байесовский метод.
4. Создайте 5 сообщений (текстов) и проверьте их на спамность вашим классификатором.
5. Повторите пункты 3 и 4, применяя сглаживание в методе. При этом добавьте два слова, которые не встречались в спам-сообщениях, и в индикаторы, и в два тестовых сообщения.
6. Сравните результаты и сделайте выводы

In [8]:
pip install easyimap

In [9]:
import easyimap
import re
from collections import Counter
import imaplib as imap
import numpy as np
import pandas as pd

In [3]:
def adgactive(text):
    ends = ['ая','яя','ой','ей','ую','юю','ый','ой','ий','ого','его','ому','ему','ым','им','ом','ем','ое','ее','ые','ие','ых','их','ыми','ими']
    for w in text:
        if w[-2:] in ends or w[-3:] in ends:
            #print(w, w[-2:], w[-3:])
            text.remove(w)  
    return text

In [2]:
def read_spam_ham():
    host = "imap.gmail.com"
    user = "kofanova.yana@gmail.com"
    password = "kofanova.yana17082002"
    imapper = easyimap.connect(host, user, password, mailbox = '[Gmail]/&BCEEPwQwBDw-')
    mail_ids = imapper.listids(limit=10)
    spam = []
    for i in mail_ids:
        email = imapper.mail(i)
        spam.append({
            'text': adgactive([word for word in re.compile('[а-яА-ЯёЁ]{2,}').findall(email.body)]),
            'key': 'spam' 
        }
        )
    #print(spam)
    imapper.change_mailbox('INBOX')
    mail_ids = imapper.listids(limit=10)
    ham = []
    for i in mail_ids:
        email = imapper.mail(i)
        ham.append({
            'text': adgactive([word for word in re.compile('[а-яА-ЯёЁ]{2,}').findall(email.body)]),
            'key': 'ham' 
        }
        )
    #print('\n')
    #print(ham)
    frame = pd.DataFrame(spam + ham)
    frame.to_csv('spam.csv')
    #print(frame)
    return spam, ham

In [4]:
def comm(text):
    indicators = []
    most_common_words = Counter(text).most_common()
    for word in most_common_words:
        if len(word[0]) >= 5:
            indicators.append(word[0])
    return indicators[:7]

def to_list(spam):
    emails = []
    for email in spam:
        for word in email['text']:
            emails.append(word)
    return emails

In [5]:
def bayes_metod(email, spam, ham, indicators):
    email = message.split(" ")
    email_spam = []
    P = 0
    for word in email:
        if word in indicators:
            email_spam.append(word)  
    print(email_spam)
    count_spam = 0
    count_ham = 0
    
    if len(email_spam) == 1:        
        for word in to_list(spam):
            if word in indicators: 
                count_spam += 1
        P_spam = count_spam/len(to_list(spam))
        
        for word in to_list(ham):
            if word in indicators: 
                count_ham += 1
        P_ham = count_ham/len(to_list(ham))
        
    else:
        k = 1
        P_spam = 1
        P_ham = 1
        count_spam = 0
        count_ham = 0
        
        for indicator in email_spam:
            for text in ham:
                if indicator in text['text']: count_ham += 1
            P_ham *= count_ham / len(to_list(spam))
                      
            for text in spam:
                if indicator in text['text']: count_spam += 1
            P_spam *= count_spam / len(to_list(ham))
                       
    P = P_spam/(P_spam + P_ham) 
    return P

In [6]:
def bayes_metod_sg(message, spam, ham, indicators):
    email = message.split(" ")
    email_spam = []
    P = 0
    for word in email:
        if word in indicators:
            email_spam.append(word)  
    print(email_spam)
    count_spam = 0
    count_ham = 0
    
    if len(email_spam) == 1:        
        for word in to_list(spam):
            if word in indicators: 
                count_spam += 1
        P_spam = count_spam/len(to_list(spam))
        
        for word in to_list(ham):
            if word in indicators: 
                count_ham += 1
        P_ham = count_ham/len(to_list(ham))
        
    else:
        k = 1
        P_spam = 1
        P_ham = 1
        count_spam = 0
        count_ham = 0
        
        for indicator in email_spam:
            for text in ham:
                if indicator in text['text']: 
                    count_ham += 1
            P_ham *= ((k + count_ham) / (2*k + len(ham)))
            
            for text in spam:
                if indicator in text['text']: 
                    count_spam += 1
            P_spam *= ((k + count_spam) / (2*k + len(spam)))
            
    P = P_spam/(P_spam + P_ham) 
    return P

In [18]:
def bayes_metod_sg_sp(email, spam, ham, indicators):
    #email = message.split(" ")
    email_spam = []
    P = 0
    for word in email:
        if word in indicators:
            email_spam.append(word)  
    print(email_spam)
    count_spam = 0
    count_ham = 0
    
    if len(email_spam) == 1:        
        for word in to_list(spam):
            if word in indicators: 
                count_spam += 1
        P_spam = count_spam/len(to_list(spam))
        
        for word in to_list(ham):
            if word in indicators: 
                count_ham += 1
        P_ham = count_ham/len(to_list(ham))
        
    else:
        k = 1
        P_spam = 1
        P_ham = 1
        count_spam = 0
        count_ham = 0
        
        for indicator in email_spam:
            for text in ham:
                if indicator in text['text']: 
                    count_ham += 1
            P_ham *= ((k + count_ham) / (2*k + len(ham)))
            
            for text in spam:
                if indicator in text['text']: 
                    count_spam += 1
            P_spam *= ((k + count_spam) / (2*k + len(spam)))
            
    P = P_spam/(P_spam + P_ham) 
    return P

In [26]:
messages = [
    'Пользователь Алиса добавила вас в заявку',
    'В данном документе можно ошибиться',
    'Договор необходимо заключить в ближайшие два дня, данные будут обработаны',
    'Поступит перевод на ваш расчетный счет указанный в комментарии за выполние работб каналы по заработку будут ниже',
    'Отчет депозитария письмо 1'
]
spam, ham = read_spam_ham()
indicators = comm(to_list(spam))
print('Индикаторы')
print(indicators)
print("\nБез сглаживания")
for message in messages:
    print(bayes_metod(message, spam, ham, indicators))
indicators.append('запись') 
indicators.append('решение')
messages.append('Было принято решение отстранить вас от работы')
messages.append('Вашу учетную запись взломали')
print("\nСглаживание")
for message in messages:
    print(bayes_metod_sg(message, spam, ham, indicators))
message_spam = [s['text'] for s in spam]
print('\nДля спама настоящего')
for message in message_spam:
    print(bayes_metod_sg_sp(message, spam, ham, indicators))

Индикаторы
['письмо', 'настройки', 'каналы', 'Пользователь', 'упомянул', 'комментарии', 'документе']

Без сглаживания
['Пользователь']
0.9256026187878187
['документе']
0.9256026187878187
[]
0.5
['комментарии', 'каналы']
1.0
['письмо']
0.9256026187878187

Сглаживание
['Пользователь']
0.9256026187878187
['документе']
0.9256026187878187
[]
0.5
['комментарии', 'каналы']
0.984182776801406
['письмо']
0.9256026187878187
['решение']
0.9256026187878187
['запись']
0.9256026187878187

Для спама настоящего
['настройки', 'каналы', 'каналы']
0.9930178432893716
['каналы', 'каналы']
0.963855421686747
[]
0.5
['Пользователь', 'упомянул', 'комментарии', 'документе', 'письмо', 'настройки']
0.999999803337609
['Пользователь', 'упомянул', 'комментарии', 'документе', 'письмо', 'настройки', 'письмо']
0.9999999803337575
['Пользователь', 'упомянул', 'комментарии', 'документе', 'письмо', 'настройки']
0.999999803337609
['Пользователь', 'упомянул', 'комментарии', 'документе', 'письмо', 'настройки', 'письмо']
0.9999

В ходе лабораторной работы был получен спам, а также все письма, из почтового ящика Gmail и записаны в CSV-файл. Из всех писем были удалены прилагательные. В спаме были подсчитаны наиболее встречающиеся слова, из которых оставлены только 7, как индикаторы спама. После составили письма для проверки на спамность. Был использован наивный баесовский метод без и с сглаживанием. Без сглаживания были получены менее точные значения, так для 2 слов вероятность равна ровно 1, когда со сглаживанием ~ 0.979.